In [166]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [167]:
import time
import numpy as np
import numba 
import cython
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


# Somebody's naive_convolve test

### Cython with numpy

In [2]:
%%cython
# cython: infer_types=True

cimport cython
import numpy as np

ctypedef fused my_type:
    int
    double
    long

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing
def naive_convolve(my_type [:,::1] f, my_type [:,::1] g):
    # f is an image and is indexed by (v, w)
    # g is a filter kernel and is indexed by (s, t),
    #   it needs odd dimensions
    # h is the output image and is indexed by (x, y),
    #   it is not cropped
    if g.shape[0] % 2 != 1 or g.shape[1] % 2 != 1:
        raise ValueError("Only odd dimensions on filter supported")

    # smid and tmid are number of pixels between the center pixel
    # and the edge, ie for a 5x5 filter they will be 2.
    #
    # The output size is calculated by adding smid, tmid to each
    # side of the dimensions of the input image.
    
    vmax = f.shape[0]
    wmax = f.shape[1]
    smax = g.shape[0]
    tmax = g.shape[1]
    smid = smax // 2
    tmid = tmax // 2
    xmax = vmax + 2*smid
    ymax = wmax + 2*tmid

    if my_type is int:
        dtype = np.intc
    elif my_type is double:
        dtype = np.double
    else:
        dtype = np.long
        
    h_np = np.zeros([xmax, ymax], dtype=dtype)
    cdef my_type [:,::1] h = h_np
    
    # Do convolution
    cdef my_type value
    for x in range(xmax):
        for y in range(ymax):
            # Calculate pixel value for h at (x,y). Sum one component
            # for each pixel (s, t) of the filter g.
            s_from = max(smid - x, -smid)
            s_to = min((xmax - x) - smid, smid + 1)
            t_from = max(tmid - y, -tmid)
            t_to = min((ymax - y) - tmid, tmid + 1)
            value = 0
            for s in range(s_from, s_to):
                for t in range(t_from, t_to):
                    v = x - smid + s
                    w = y - tmid + t
                    value += g[smid - s, tmid - t] * f[v, w]
            h[x, y] = value
    return h_np

In [124]:
N = 600
f = np.arange(N*N, dtype=np.int).reshape((N,N))
g = np.arange(81, dtype=np.int).reshape((9, 9))

In [184]:
# Pure Python version
%timeit naive_convolve(f, g)

18.2 s ± 920 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [129]:
# Cython version with no change in code -- only some slight improvment
%timeit naive_convolve(f, g)

12.9 s ± 154 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [132]:
# Cython version with some types added -- manual version
%timeit naive_convolve(f, g)

9.65 s ± 494 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [158]:
# Cython version with memory views
%timeit naive_convolve(f, g)

118 ms ± 2.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [160]:
# Cython version with memory views and deactivated boundary checking & negative indexing
%timeit naive_convolve(f, g)

24.7 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [162]:
# Cython version with memory views, deactivated boundary checking & negative indexing, and contiguous memory
%timeit naive_convolve(f, g)

21.9 ms ± 519 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [168]:
# Cython version with memory views, deactivated boundary checking & negative indexing, contiguous memory, and inferred and fused types
%timeit naive_convolve(f, g)

21.8 ms ± 381 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Numba magic

In [183]:
import numpy as np
def naive_convolve(f, g):
    # f is an image and is indexed by (v, w)
    # g is a filter kernel and is indexed by (s, t),
    #   it needs odd dimensions
    # h is the output image and is indexed by (x, y),
    #   it is not cropped
    if g.shape[0] % 2 != 1 or g.shape[1] % 2 != 1:
        raise ValueError("Only odd dimensions on filter supported")
    # smid and tmid are number of pixels between the center pixel
    # and the edge, ie for a 5x5 filter they will be 2.
    #
    # The output size is calculated by adding smid, tmid to each
    # side of the dimensions of the input image.
    vmax = f.shape[0]
    wmax = f.shape[1]
    smax = g.shape[0]
    tmax = g.shape[1]
    smid = smax // 2
    tmid = tmax // 2
    xmax = vmax + 2*smid
    ymax = wmax + 2*tmid
    # Allocate result image.
    h = np.zeros([xmax, ymax], dtype=f.dtype)
    # Do convolution
    for x in range(xmax):
        for y in range(ymax):
            # Calculate pixel value for h at (x,y). Sum one component
            # for each pixel (s, t) of the filter g.
            s_from = max(smid - x, -smid)
            s_to = min((xmax - x) - smid, smid + 1)
            t_from = max(tmid - y, -tmid)
            t_to = min((ymax - y) - tmid, tmid + 1)
            value = 0
            for s in range(s_from, s_to):
                for t in range(t_from, t_to):
                    v = x - smid + s
                    w = y - tmid + t
                    value += g[smid - s, tmid - t] * f[v, w]
            h[x, y] = value
    return h

In [185]:
pairwise_numba = numba.jit(naive_convolve)

In [189]:
# Numba version
%timeit pairwise_numba(f, g)

74.4 ms ± 5.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Numba is ~3X slower than best cython version. It's up to one's decision if best cython code is worth the effort.

# My test 1 -- compute the mean of an array

In [168]:
def get_mean(arr):
    sz = len(arr)
    u = 0
    for x in arr:
        u += x / sz
    return u

In [169]:
%%cython

cimport cython
import numpy as np

@cython.cdivision(True)     # Use C division
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing
@cython.infer_types(True)   # Activate infer type
def get_mean_cython(double [::1] arr):
    sz = arr.shape[0]
    cdef double u = 0
    for i in range(sz):
        u += arr[i] / sz
    return u

In [170]:
get_mean_numba = numba.jit(get_mean)

In [171]:
# Define arrays
arr = np.random.rand(5000000)

In [172]:
# Check the results
get_mean(arr), np.mean(arr), get_mean_numba(arr), get_mean_cython(arr)

(0.4999144199078945,
 0.4999144199079061,
 0.4999144199078945,
 0.4999144199078945)

In [173]:
# Pure python
%timeit get_mean(arr)

2.49 s ± 149 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [174]:
# Numpy
%timeit np.mean(arr)

7.35 ms ± 415 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [175]:
# Numba
%timeit get_mean_numba(arr)

7.12 ms ± 42.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [176]:
# Cython
%timeit get_mean_cython(arr)

7.08 ms ± 72.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Numpy, Numba, and Cython all give ~300X boost in speed!

# My test 2 -- compute the mean of an array, 2nd version

In [177]:
def get_mean2(arr):
    return sum(arr) / len(arr)

In [178]:
# Pure python
%timeit get_mean2(arr)

803 ms ± 9.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [180]:
get_mean_numba2 = numba.jit(get_mean2)

# Numba
%timeit get_mean_numba2(arr)

785 ms ± 6.07 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [181]:
# Cython
%timeit get_mean_cython(arr)

7.04 ms ± 28 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Note there's not much difference between pure Python and Numba versions! This means numba is not very efficient in taking care of Python's sum function.

# My test 2 -- compute the mean and standard devision of weighted array

In [182]:
def get_mean_and_std_weighted(arr, weights):
    # Compute total weight
    tw = 0
    for w in weights:
        tw += w
    
    # Compute mean
    u = 0
    for x, w in zip(arr, weights):
        u += (x * w) / tw
        
    # Compute std
    s = 0
    for x, w in zip(arr, weights):
        tmp = (x - u)
        s += w * tmp * tmp / tw
    s = np.sqrt(s)
    return u, s

In [183]:
%%cython

cimport cython
import numpy as np

@cython.cdivision(True)     # Use C division
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing
@cython.infer_types(True)   # Activate infer type
def get_mean_and_std_weighted_cython(double [::1] arr, double [::1] weights):
    # Compute total weight
    cdef double tw = 0
    for i in range(weights.shape[0]):
        tw += weights[i]
    
    # Compute mean
    cdef double u = 0
    for i in range(weights.shape[0]):
        u += (arr[i] * weights[i]) / tw
        
    # Compute std
    cdef double s = 0
    cdef double tmp = 0
    for i in range(weights.shape[0]):
        tmp = (arr[i] - u)
        s += weights[i] * tmp * tmp / tw
    s = np.sqrt(s)
    return u, s

In [184]:
get_mean_and_std_weighted_numba = numba.jit(get_mean_and_std_weighted)

In [185]:
# Define arrays
arr = np.random.rand(500000)
weights = np.random.rand(500000)

In [186]:
# Check the results
get_mean_and_std_weighted(arr, weights), get_mean_and_std_weighted_numba(arr, weights), get_mean_and_std_weighted_cython(arr, weights)

((0.49957791713802724, 0.2887419641056903),
 (0.49957791713802724, 0.2887419641056903),
 (0.49957791713802724, 0.2887419641056903))

In [187]:
# Pure python
%timeit get_mean_and_std_weighted(arr, weights)

701 ms ± 9.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [188]:
# Numba
%timeit get_mean_and_std_weighted_numba(arr, weights)

2.34 ms ± 84.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [189]:
# Cython
%timeit get_mean_and_std_weighted_cython(arr, weights)

2.25 ms ± 46 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Again, both Numba and Cython give ~300X speed boosts! Also, also note if infer_type = True, Cython can run ~10X faster if not all variables' types are spcified.

# Draft